In [18]:
from sklearn.cluster import KMeans
from transformers import BertTokenizer, BertModel
import torch
import re
import pandas as pd
from datetime import datetime

def getRightNow():
    now = datetime.now()
    hms = now.strftime("%H:%M:%S")
    return hms
print("Imports {}".format(getRightNow()))

Imports 10:26:01


In [ ]:
def number_to_letter(num):
    # Why letters as labels rather than numbers? 
    # Because numbers sort of imply an order
    # Letters do also, but less so.
    x = num
    if num <= 0:
        raise ValueError("Input must be a positive integer.")
    
    letters = ""
    while num > 0:
        num -= 1
        # Map the remainder to the corresponding letter
        letters = chr(num % 26 + ord('A')) + letters  
        # Divide by 26 to move to the next digit
        num //= 26
        
    return letters

In [23]:
def ETL(outfile, infile):
    def makePrettyRow(x,  dupCount, ith):
        lookUpId = number_to_letter(ith)
        theImportantStuff = "{}|{}|{}|{}".format(x["name"], x["id"],x["activity"],x["desc"])
        n = len(theImportantStuff)
        # lookUpId = row look up...  so 'XYZ' will point to 'some description'
        # n = char count --- curious how length affects the BERT model
        # dupCount --- might want to weight things somewhat depending on how duplicated they are ( assumption = more dups is more good )
        return "{}|{}|{}\n".format(  n, dupCount, theImportantStuff)
    
    def nothingIsEmpty(ary): 
        # Lots of the cells are empty and illformed - skip any row with an empty cell
        isOk = len(ary[0]) > 0 and  len(ary[1]) > 0 and len(ary[2]) > 0 and len(ary[3]) > 0 
        return isOk 
    
    def convert_numbers_to_x(string):
        # Convert "blah blah size 6" or "blah blah size 16" to "blah blah size x"
        # Needed to squish near duplicated lines
        return re.sub(r'\d+', 'x', string)

    descriptions = [] 
    seen = {} 
    products = [] 
    with open(infile, 'r') as file:
        for line in file:
            pieces = line.replace('"', '').strip().split('|')
            if len(pieces) == 4:
                pieces[3] = convert_numbers_to_x(pieces[3])
                if nothingIsEmpty(pieces) == True: 
                    x = {
                        "name":pieces[0],
                        "id":pieces[1],
                        "activity":pieces[2],
                        "desc":pieces[3]
                    }
                    if x['desc'] not in seen: 
                        seen[x["desc"]] = 1
                        products.append(x)
                    else:
                        seen[x["desc"]] += 1
                        
                        
    with open(outfile, 'w') as file:
        # for x in products[0:3]:
        i = 0
        for x in products:
            i += 1
            r = makePrettyRow(x,  seen[x["desc"]], i)
            file.write(r)
            
    
#     data = pd.read_excel(infile)
#     data = data.applymap(lambda x: str(x).replace('\"','').replace('"', '').replace('|', '').replace('\n', '').replace('\\', '').replace('"','').replace('™', '').replace('®', '').replace(' -','')) 
#     # Drop rows with duplicate values in the last column
#     data.drop_duplicates(subset=data.columns[-1], keep=False, inplace=True)
#     # Convert numbers to 'x'
#     data = data.applymap(convert_numbers_to_x)
#     # Convert DataFrame to PIPE delimited flat file
#     output = data.apply(lambda x: '|'.join(x.astype(str)), axis=1)
#     # Write the output to a file
#     output.to_csv(outfile, index=False, header=False)
#     print("{} => {} @ {}".format( infile, outfile, getRightNow()))

#     for x in output[0:3]:
#         pieces = x.split("|")
#         if len(pieces) == 4: 
#             d = {
#                 "name":pieces[0],
#                 "id":pieces[1],
#                 "activity":pieces[2],
#                 "desc":pieces[3]
#             }
#             print(d)
ETL("output_file2.txt", "Data_June_16_redacted.txt")

    


In [8]:
# List of product descriptions
product_descriptions = [
    "High-quality leather wallet for men",
    "Elegant and stylish women's handbag",
    "Durable and lightweight hiking backpack",
    "Soft and comfortable cotton t-shirt",
    "Premium stainless steel kitchen knife set"
]

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode the product descriptions
encoded_inputs = tokenizer(product_descriptions, padding=True, truncation=True, return_tensors='pt')

# Get the embeddings using BERT model
with torch.no_grad():
    outputs = model(**encoded_inputs)

# Get the sentence-level embeddings (CLS token)
embeddings = outputs.last_hidden_state[:, 0, :]

# Convert embeddings to numpy array
embeddings = embeddings.numpy()

# Perform clustering using K-Means algorithm
num_clusters = 2  # Number of clusters to create
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
cluster_labels = kmeans.fit_predict(embeddings)

# Print the cluster labels for each product description
for i, desc in enumerate(product_descriptions):
    print(f"Product: {desc}\tCluster: {cluster_labels[i]}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Product: High-quality leather wallet for men	Cluster: 1
Product: Elegant and stylish women's handbag	Cluster: 0
Product: Durable and lightweight hiking backpack	Cluster: 1
Product: Soft and comfortable cotton t-shirt	Cluster: 0
Product: Premium stainless steel kitchen knife set	Cluster: 0
